In [1]:
# Compatibility shim for renamed module
import sys
import app.services.ocr.paddle_ocr_service as _paddle_ocr_service
sys.modules['app.services.ocr.paddle_ocr'] = _paddle_ocr_service


In [2]:
# DI providers setup: use centralized dependencies from app.core.deps
from app.core.deps import (
    get_config,
    get_ocr_service,
    get_image_preprocessor,
    get_line_preprocessor,
    get_extractor,
    clear_cached_providers,
)

# Initialize singletons/factories
config = get_config()
ocr_service = get_ocr_service()
image_preprocessor = get_image_preprocessor()
line_preprocessor = get_line_preprocessor()
lab_table_extractor = get_extractor()

# Backward-compat variable aliases (keep existing cells working)
ocr = ocr_service
image_pre = image_preprocessor
line_pre = line_preprocessor
extractor = lab_table_extractor

print("DI ready:")
print("  config=", type(config).__name__)
print("  ocr_service=", type(ocr_service).__name__)
print("  image_preprocessor=", type(image_preprocessor).__name__)
print("  line_preprocessor=", type(line_preprocessor).__name__)
print("  lab_table_extractor=", type(lab_table_extractor).__name__)


/home/aidan/.local/lib/python3.10/site-packages/paddle/utils/cpp_extension/extension_utils.py:718: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)
Creating model: ('PP-LCNet_x1_0_doc_ori', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/aidan/.paddlex/official_models/PP-LCNet_x1_0_doc_ori`.
Creating model: ('PP-LCNet_x1_0_doc_ori', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/aidan/.paddlex/official_models/PP-LCNet_x1_0_doc_ori`.
Creating model: ('UVDoc', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/aidan/.paddlex/official_models/UVDoc`.
Creating model: ('UVDoc', None)
Model files already exist. Using cached

DI ready:
  config= AppConfig
  ocr_service= PaddleOCRService
  image_preprocessor= ImagePreprocessor
  line_preprocessor= LinePreprocessor
  lab_table_extractor= LabTableExtractor


## 라이브러리 경로 설정 (GLIBCXX 문제 해결)

PyMuPDF 패키지의 `libtesseract.so.5`가 `GLIBCXX_3.4.31`을 요구하는데, 시스템 libstdc++는 3.4.30까지만 지원합니다.

### ✅ 해결 방법

**옵션 1: Jupyter 재시작 (권장)**
```bash
# 프로젝트 루트에서
./start_jupyter.sh
```

**옵션 2: VS Code에서 현재 세션**
아래 셀을 실행 후, 커널 재시작 필요
- Kernel → Restart Kernel 클릭
- 또는 Ctrl+Shift+P → "Jupyter: Restart Kernel"

In [3]:
# Conda 환경의 libstdc++.so.6을 RTLD_GLOBAL로 먼저 로드
import ctypes
import os
import sys

conda_lib = os.path.join(sys.prefix, 'lib', 'libstdc++.so.6')

# 환경변수가 이미 설정되어 있는지 확인
ld_path = os.environ.get('LD_LIBRARY_PATH', '')
conda_lib_dir = os.path.join(sys.prefix, 'lib')

if conda_lib_dir in ld_path.split(':'):
    print(f"✅ LD_LIBRARY_PATH가 이미 설정되어 있습니다: {conda_lib_dir}")
    print("   → ./start_jupyter.sh 로 시작한 경우 정상입니다.")
else:
    print(f"⚠️  LD_LIBRARY_PATH가 설정되지 않았습니다.")
    print(f"   현재 커널에서는 GLIBCXX 에러가 발생할 수 있습니다.")
    print()
    print("해결 방법:")
    print("  1) 터미널에서 ./start_jupyter.sh 실행")
    print("  2) 또는 아래 명령으로 환경변수 설정 후 커널 재시작:")
    print(f"     export LD_LIBRARY_PATH={conda_lib_dir}:$LD_LIBRARY_PATH")
    
# libstdc++.so.6 로드 시도 (환경변수가 없을 때 fallback)
try:
    ctypes.CDLL(conda_lib, mode=ctypes.RTLD_GLOBAL)
    print(f"\n✅ Fallback: Conda libstdc++ 로드 완료")
    print(f"   → 커널 재시작 후 다시 실행하면 정상 작동할 수 있습니다.")
except Exception as e:
    print(f"\n⚠️  Fallback 로드 실패: {e}")
    print(f"   심볼릭 링크 확인: ls -la {conda_lib}")

⚠️  LD_LIBRARY_PATH가 설정되지 않았습니다.
   현재 커널에서는 GLIBCXX 에러가 발생할 수 있습니다.

해결 방법:
  1) 터미널에서 ./start_jupyter.sh 실행
  2) 또는 아래 명령으로 환경변수 설정 후 커널 재시작:
     export LD_LIBRARY_PATH=/home/aidan/miniconda3/envs/meow-chat/lib:$LD_LIBRARY_PATH

✅ Fallback: Conda libstdc++ 로드 완료
   → 커널 재시작 후 다시 실행하면 정상 작동할 수 있습니다.


## 여러개 파일 연속 처리 테스트

In [4]:
img_paths = [
    os.path.abspath('tests/notebooks/ocr/assets/images/20241121_0.png'),
    os.path.abspath('tests/notebooks/ocr/assets/images/20241113.png'),
    os.path.abspath('tests/notebooks/ocr/assets/images/20241121_1.png'),
]

In [5]:
import os

from app.services.analysis.image_preprocessor import ImagePreprocessor, Settings as IPS
from app.services.ocr.paddle_ocr import PaddleOCRService
from app.services.analysis.line_preprocessor import LinePreprocessor, Settings as LPSettings
from app.services.analysis.reference.unit_lexicon import get_unit_lexicon
import app.services.analysis.lab_table_extractor as lte
import importlib
import app.services.analysis.reference.code_lexicon as cl

lte = importlib.reload(lte)
LabTableExtractor = lte.LabTableExtractor

Settings = lte.Settings

ip = ImagePreprocessor(IPS(debug=True))

ocr = PaddleOCRService(lang="korean")

unit_lex = get_unit_lexicon()  # 전역 유닛 렉시콘(캐시)

lp = LinePreprocessor(LPSettings(
    order="x_left",
    alpha=0.7,
    debug=True,
    normalize_units_first_pass=True,  # 분리 직후 1차 단위 정규화 
), unit_lexicon=unit_lex)


cl = importlib.reload(cl)
code_lexicon = cl.get_code_lexicon()  # 캐시 사용 (참조 데이터 변경 없을 때 추천)

extractor = LabTableExtractor(
    settings=Settings(use_llm=True, debug=True),  # 정책 미달 시에만 LLM 백업 사용
    lexicon=code_lexicon,
    resolver=cl.resolve_code,  # ← 중요: 최신 resolve_code를 명시 주입
    api_key=os.getenv("OPENAI_API_KEY"),
    llm_model="gpt-4.1-mini",
)

Creating model: ('PP-LCNet_x1_0_doc_ori', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/aidan/.paddlex/official_models/PP-LCNet_x1_0_doc_ori`.
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/aidan/.paddlex/official_models/PP-LCNet_x1_0_doc_ori`.
Creating model: ('UVDoc', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/aidan/.paddlex/official_models/UVDoc`.
Creating model: ('UVDoc', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/aidan/.paddlex/official_models/UVDoc`.
Creating model: ('PP-LCNet_x1_0_textline_ori', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/aidan/.paddlex/official_models/PP-LCNet_x1_0_textline_ori`.
Creating model: ('PP-LCNet_x1_0_textline_ori', None)

In [6]:
print(f"[전처리+OCR] 총 {len(img_paths)}개 이미지 처리 시작")


# 추출 결과 누적 리스트
results = []


for p in img_paths:
    print(f"\n[작업 시작] {p}")
    try:
        with open(p, 'rb') as f:
            _raw_bytes = f.read()


        # 클래스 기반 전처리 실행 (디버그 로그 포함: 데스큐 각도, 용량 변화 등)
        _pre_bytes = ip.process_bytes(_raw_bytes, debug=True)
        print(f"✅ ImagePreprocessor 로 전처리 완료: {p} (bytes: {len(_pre_bytes) if _pre_bytes else 'None'})")


        # OCR 실행 (Pydantic 모델 반환)
        ocr_env = ocr.run_ocr_from_bytes(_pre_bytes)
        items = (getattr(ocr_env, 'data', None).items
                  if hasattr(ocr_env, 'data') and hasattr(ocr_env.data, 'items')
                  else None)
        items = items if isinstance(items, list) else []


        # 라인 정렬 및 값/단위 분리 (OCR items 전체를 입력)
        lined = lp.extract_and_group_lines(items)


        # 추출
        result = extractor.extract(lined)
        # 누적
        results.append(result)
    except Exception as e:
        print(f"⚠️ 처리 실패: {p} - {e}")

[전처리+OCR] 총 3개 이미지 처리 시작

[작업 시작] /home/aidan/work/meow-chat/tests/notebooks/ocr/assets/images/20241121_0.png
🏥 이미지 전처리 시작 (ImagePreprocessor)
🔁 EXIF 회전 교정 적용 시도
📏 원본 이미지 크기: (1179, 1568)
🧰 실행 단계: 1 → upscale_min_resolution
[pipeline] -> upscale_min_resolution(min_long_edge=1920)
[pipeline]    size (1179, 1568) → (1444, 1920), mode RGBA → RGBA
💾 저장 형식: PNG(optimize=True)
📐 최종 이미지 크기: (1444, 1920)
🔄 크기 변화: (1179, 1568) → (1444, 1920)
💾 용량 변화: 2,424,766 bytes → 2,911,659 bytes (120.1%)
✅ 전처리 완료
✅ ImagePreprocessor 로 전처리 완료: /home/aidan/work/meow-chat/tests/notebooks/ocr/assets/images/20241121_0.png (bytes: 2911659)
📐 최종 이미지 크기: (1444, 1920)
🔄 크기 변화: (1179, 1568) → (1444, 1920)
💾 용량 변화: 2,424,766 bytes → 2,911,659 bytes (120.1%)
✅ 전처리 완료
✅ ImagePreprocessor 로 전처리 완료: /home/aidan/work/meow-chat/tests/notebooks/ocr/assets/images/20241121_0.png (bytes: 2911659)
❌ rec_texts를 찾을 수 없습니다.
[DEBUG] tokens_with_geometry: count=0
[]

[작업 시작] /home/aidan/work/meow-chat/tests/notebooks/ocr/assets/imag

In [7]:
import json

print("병합 전")
print(json.dumps(results, ensure_ascii=False, indent=2))

병합 전
[
  {
    "hospital_name": "",
    "client_name": "",
    "patient_name": "",
    "inspection_date": "",
    "header_shape": "",
    "tests": []
  },
  {
    "hospital_name": "",
    "client_name": "",
    "patient_name": "",
    "inspection_date": "",
    "header_shape": "",
    "tests": []
  },
  {
    "hospital_name": "",
    "client_name": "",
    "patient_name": "",
    "inspection_date": "",
    "header_shape": "",
    "tests": []
  }
]


In [8]:
# manager 기반 병합: 결과 리스트(results)를 표준 MergeEnvelope로 병합 후 JSON 출력
import os
from app.core.deps import get_pipeline_manager

# manager 인스턴스가 없으면 DI 프로바이더로 생성
try:
    manager
except NameError:
    manager = get_pipeline_manager()

print("병합 후 (manager.merge_extractions)")
merged_env = manager.merge_extractions(results)
try:
    print(merged_env.model_dump_json(indent=2, ensure_ascii=False))
except Exception:
    # pydantic 2.x 호환 불가 환경 대비 안전 출력
    print(getattr(merged_env, 'model_dump', lambda **_: merged_env)())


병합 후 (manager.merge_extractions)
{
  "stage": "merge",
  "data": {
    "merged": []
  },
  "meta": {
    "pruned_empty": 3,
    "merged_len": 0,
    "before_dedup": 0,
    "after_dedup": 0
  },
  "version": "1.0"
}


### OCRPipelineManager 개요

- 목적: 노트북의 처리 단계를 3개의 기능 단위로 모듈화하여 재사용(특히 MCP 서버) 가능하게 함

- 3단계 인터페이스(입력→출력):

  1) image_to_ocr(bytes) → ocr_result

  2) ocr_to_extraction(ocr_result) → extraction(dict)

  3) merge_extractions(list[dict]) → json(str)



간단 계약(Contract)

- image_to_ocr

  - 입력: 원본 이미지 bytes

  - 옵션: do_preprocess=True일 때 내부 전처리(ImagePreprocessor.process_bytes) 수행

  - 출력: PaddleOCRService.run_ocr_from_bytes 결과(리스트/튜플 등 OCR 결과 객체)

- ocr_to_extraction

  - 입력: OCR 결과(위 1단계 출력)

  - 처리: LinePreprocessor.extract_and_group_lines → LabTableExtractor.extract

  - 출력: 검사표 추출 결과 dict

- merge_extractions

  - 입력: 여러 개의 extraction(dict)의 리스트

  - 처리: (code, unit) 중복 제거 및 연속 병합 규칙 적용

  - 출력: ensure_ascii=False, indent=2의 JSON 문자열



MCP 서버 적합성

- 각 단계가 순수 함수 스타일로 분리되어, 서버 핸들러에서 단계별 엔드포인트/툴로 노출이 용이

- 의존성(전처리기/라인 정렬/추출기)은 생성자 주입으로 테스트/교체가 쉬움

- 결과 직렬화(JSON) 인터페이스를 제공해 도구 체인 간 전달이 간단함

In [9]:
# 매니저 사용 예시: img_paths를 순차 처리하여 병합 JSON 출력 (Pydantic 모델 반환 표준화)
import os
from app.core.deps import get_pipeline_manager

# 샘플 이미지 경로
img_paths = [
    os.path.abspath('tests/notebooks/ocr/assets/images/20241121_0.png'),
    os.path.abspath('tests/notebooks/ocr/assets/images/20241113.png'),
    os.path.abspath('tests/notebooks/ocr/assets/images/20241121_1.png'),
]

# 진행 상황 출력 콜백

def print_progress(evt):
    stage = evt.get('stage')
    status = evt.get('status')
    extra = {k: v for k, v in evt.items() if k not in ('stage', 'status', 'ts', 'result', 'json')}
    print(f"[progress] {stage}.{status} {extra}")

# DI 프로바이더로 매니저 생성
manager = get_pipeline_manager(progress_cb=print_progress)

# img_paths에 있는 이미지를 순차 처리
extractions = []
print(f"[전처리+OCR] 총 {len(img_paths)}개 이미지 순차 처리 시작")
for p in img_paths:
    print(f"\n[작업 시작] {p}")
    try:
        with open(p, 'rb') as f:
            b = f.read()
        # 1) 이미지 -> OCR 결과(모델)
        ocr_env = manager.image_to_ocr(
            b,
            do_preprocess=True,
            preprocess_kwargs={'debug': True},
        )
        # 2) OCR 결과 -> 추출(모델)
        extract_env = manager.ocr_to_extraction(ocr_env)
        tests = None
        try:
            tests = extract_env.data.get('tests') if hasattr(extract_env, 'data') else None
        except Exception:
            tests = None
        tests_len = len(tests) if isinstance(tests, list) else None
        print(f"✅ extraction 완료: tests={tests_len}")
        # 병합 입력은 순수 추출 dict 리스트
        extractions.append(extract_env.data if hasattr(extract_env, 'data') else {})
    except Exception as e:
        print(f"⚠️ 처리 실패: {p} - {e}")

# 3) 여러 추출 결과 병합 후 모델 출력
print("\n병합 결과(JSON):")
merged_env = manager.merge_extractions(extractions)
try:
    print(merged_env.model_dump_json(indent=2, ensure_ascii=False))
except Exception:
    # pydantic 2.x 호환 불가 환경을 대비한 안전 출력
    print(getattr(merged_env, 'model_dump', lambda **_: merged_env)())


[전처리+OCR] 총 3개 이미지 순차 처리 시작

[작업 시작] /home/aidan/work/meow-chat/tests/notebooks/ocr/assets/images/20241121_0.png
[progress] preprocess.start {}
🏥 이미지 전처리 시작 (ImagePreprocessor)
🔁 EXIF 회전 교정 적용 시도
📏 원본 이미지 크기: (1179, 1568)
🧰 실행 단계: 3 → flatten_transparency, normalize_mode, deskew_textlines
[pipeline] -> flatten_transparency()
[pipeline]    size (1179, 1568) → (1179, 1568), mode RGBA → RGB
[pipeline] -> normalize_mode()
[pipeline]    size (1179, 1568) → (1179, 1568), mode RGB → RGB
[pipeline] -> deskew_textlines()
[pipeline]    size (1179, 1568) → (1179, 1568), mode RGB → RGB
💾 저장 형식: PNG(optimize=True)
[pipeline]    size (1179, 1568) → (1179, 1568), mode RGB → RGB
💾 저장 형식: PNG(optimize=True)
📐 최종 이미지 크기: (1179, 1568)
🔄 크기 변화: (1179, 1568) → (1179, 1568)
💾 용량 변화: 2,424,766 bytes → 1,911,642 bytes (78.8%)
✅ 전처리 완료
[progress] preprocess.end {'bytes': 1911642}
[progress] ocr.start {}
📐 최종 이미지 크기: (1179, 1568)
🔄 크기 변화: (1179, 1568) → (1179, 1568)
💾 용량 변화: 2,424,766 bytes → 1,911,642 bytes (7